# PDF File 

In [1]:
import fitz  # PyMuPDF

# file_path : each PDF File path
file_path = '/home/csh/workspace/DACON/finance_llm/data/train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf'

doc = fitz.open(file_path)

### fitz 라이브러리 실험

In [2]:
print(doc)
print(type(doc))
print("page_count : ",end=" ")
print(doc.page_count)
print("-"*50)
print("get page : ")
print(doc[0].get_text())
print("-"*50)
print("dtype of get_text(): ",end="")
print(type(doc[0].get_text()))
print("-"*50)

Document('/home/csh/workspace/DACON/finance_llm/data/train_source/「FIS 이슈 & 포커스」 22-3호 《재정융자사업》.pdf')
<class 'pymupdf.Document'>
page_count :  9
--------------------------------------------------
get page : 
ISSUE & FOCUS
FIS 
22-3호
2022.11
.
  발행인 박용주      발행처 04637 서울특별시  중구 퇴계로 10(남대문로5가 537) 메트로타워      T 02)6908-8200      F 02)6312-8959
작성 박정수 부연구위원, 우수연 연구원    기획
·
조정 심혜인 결산정보분석부장
재정융자사업
1  들어가며
2  재정융자사업의 개념과 의의
3  2023년도 예산안 재정융자사업 현황
4  재정융자사업의 주요 현안
5  나가며

--------------------------------------------------
dtype of get_text(): <class 'str'>
--------------------------------------------------


# Splitter
PDF에서 얻은 문자열 데이터를 청크 단위로 분할

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


text = doc[0].get_text()

# 모든 페이지의 텍스트 추출
for page in doc:
    text += page.get_text()
    
# 텍스트를 chunk로 분할
chunk_size=800
chunk_overlap=50

splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

chunk_temp = splitter.split_text(text)
# Document 객체 리스트 생성
chunks = [Document(page_content=t) for t in chunk_temp]

In [5]:
chunks[0].page_content

'ISSUE & FOCUS\nFIS \n22-3호\n2022.11\n.\n  발행인 박용주      발행처 04637 서울특별시  중구 퇴계로 10(남대문로5가 537) 메트로타워      T 02)6908-8200      F 02)6312-8959\n작성 박정수 부연구위원, 우수연 연구원    기획\n·\n조정 심혜인 결산정보분석부장\n재정융자사업\n1  들어가며\n2  재정융자사업의 개념과 의의\n3  2023년도 예산안 재정융자사업 현황\n4  재정융자사업의 주요 현안\n5  나가며\nISSUE & FOCUS\nFIS \n22-3호\n2022.11\n.\n  발행인 박용주      발행처 04637 서울특별시  중구 퇴계로 10(남대문로5가 537) 메트로타워      T 02)6908-8200      F 02)6312-8959\n작성 박정수 부연구위원, 우수연 연구원    기획\n·\n조정 심혜인 결산정보분석부장\n재정융자사업\n1  들어가며\n2  재정융자사업의 개념과 의의\n3  2023년도 예산안 재정융자사업 현황\n4  재정융자사업의 주요 현안\n5  나가며\n02\n03\nFIS    ISSUE & FOCUS \n                   들어가며\nISSUE   왜 재정융자사업에 주목하는가?\n\x03\n재정융자사업은 정부가 자금을 민간의 사적 경제주체에 대해 대출해 주고 회수하는 활동을 말하며, 정부\n의 다양한 금융활동 중 직접대출과 전대 방식에 해당(재무부, 1993; 한국재정정보원, 2019)\n    - \x03\n정부의 금융활동은 직접융자·전대 외에도 금리 차액에 대한 보상(이차보전), 기업의 신용심사 및 보증(신용보'

# Vector DB
만든 Document 데이터들을 Vector DB에 저장

- **FAISS : 파시스 (Facebook AI Similarity Search)**

대량의 고차원 벡터에서 유사성 검색 및 클러스터링을 빠르고 효율적으로 수행<br>


`from_documents` 클래스 메서드는 문서 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성<br>
<파라미터>로 임베딩 함수(Embeddings)과 데이터(List[Document]) 입력 <br>
<반환값> `VectorStore`: 문서와 임베딩으로 초기화된 벡터 저장소 인스턴스

- **VectorStore란**
자연어 처리(NLP) 

및 기계 학습 분야에서 벡터 검색을 위한 효율적인 데이터 저장 및 검색 시스템

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db


db = create_vector_db(chunks)

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
db 

In [8]:
db.save_local("./save_fasiss") # 이렇게 저장하고

In [9]:
import pickle

with open("./save_fasiss/index.pkl","rb") as fs:
    data = pickle.load(fs)
    print(data)
# 불러보면 임베딩된 값으로 저장하고 있음 
# -> 이거 다시 돌리는거로 받으려면 load_local + embedding_model

(<langchain_community.docstore.in_memory.InMemoryDocstore object at 0x7f280a9f7cd0>, {0: 'ee550deb-713a-4c0a-a52c-0d20facf956f', 1: '24e7305e-b957-4210-9b96-514cd651cc21', 2: 'e4c88563-6eb3-41af-8eed-74158641d0ec', 3: '3ec90013-f679-4f1c-859c-b1b9acd3c5b8', 4: 'de856b56-962b-40d2-a6b6-3ce9ed6eb9c4', 5: '48ab05d9-7b13-4955-8a11-c9e43278cf57', 6: '01926be0-d242-4bc3-b5b2-981ac80cac93', 7: '26abb1fb-db23-44de-bed9-2342ac35df5c', 8: '8223aee2-426a-4059-b37a-77abef43b1bf', 9: 'a45855cf-0201-4fc7-a6bd-30d4c003b5d9', 10: 'e64f3fb3-59fe-4101-94e3-aa92f7ccdaab', 11: '07da717a-3873-467c-8fa4-a1717000dc1c', 12: '6f3d2ca0-21f3-4459-86c7-839dbb2a6b07', 13: '42aa123c-8bf5-4329-add4-37bee2e737e1', 14: 'd508b7de-e08c-4000-8650-1d5fcffdf2e3', 15: '0b6060be-95f9-4e23-85f6-6bb7c208fb94', 16: '0468411a-3eb7-4f1a-b5e8-5643ca3a36e0', 17: '8545f207-2d36-4ad6-9708-1abf1853ac3c', 18: '5f664cde-af29-46b8-b072-48fb74903666', 19: 'e7ad4ad5-2701-4500-8689-8fd599fe61da', 20: 'e2bfc7af-b8f6-4605-926b-43f5f54c02aa', 

# Retriever
리트리버(retriever)는 정보 검색 시스템에서 중요한 역할을 하는 컴포넌트로, 주어진 쿼리에 대해 관련 정보를 찾고 반환하는 기능을 담당 = 간단하게 **검색 도구**

"Vector stores can be used as the backbone of a retriever"<br>
"Retrievers accept a string query as input and return a list of Document's as output."

In [10]:
# Retriever 생성
retriever = db.as_retriever(search_type="mmr", # Maximum Marginal Relevance
                  search_kwargs={'k': 3, 'fetch_k': 8}) 
                # k :검색 시 선택할 최대 결과 수 (최종)
                # fetch_k : 검색을 위해 가져올 결과 수 (후보)

In [11]:
id(db)

140631844762432

In [12]:
retriever
# 다른 저장소 (ex Chroma)도 langchain의 하위 라이브러리에 저장되어있음
# 따라서 tags에 어떤 라이브러리를 사용했는지 나와있고
# vectorstore에는 FAISS 객체가 담겨있음
# search_type과 search_kwargs에 

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fe76726b340>, search_type='mmr', search_kwargs={'k': 3, 'fetch_k': 8})

In [13]:
print(id(db) == id(retriever.vectorstore))

True


In [14]:
outputs = retriever.invoke("코로나걸려서 기침이 나와")
print(len(outputs))
print(type((outputs[0])))
print(outputs[0].page_content.find("코로나"))
print(outputs[1].page_content.find("코로나"))
print(outputs[2].page_content.find("코로나"))

3
<class 'langchain_core.documents.base.Document'>
355
303
-1


# Remind

지금까지 
1. PDF 파일을 통해 먼저 문자열로 만들었고
2. 그 문자열을 특정 단위로(청크) Spilt 했고
3. 나눠진 데이터를 Embedding 모델을 통해 Vector로 만들면서
4. FAISS를 통해 Vector DB에 저장했음
5. 마지막으로 Vector DB를 통해 Retriever 객체를 만들어서 (질문-> 유사도높은 대답 후보)를 가능하게함


중간 중간에 바꿀수 있는 것들
- **PDF를 어떤 라이브러리를 통해서 문자열로 바꿀지**
- **어떤 단위로 문서를 분리할지**
- **어떤 임베딩 모델을 사용할지**
- **어떤 Vector DB를 사용할지**
- **어떤 방식으로 Vector를 찾을지 (파라미터)**

### 추가 생각
PDF의 테이블, 표, 그림으로 되어있는 자료는 불러올 때 인식하기 어려운 경우가 존재함

In [1]:
import pandas as pd
from tqdm import tqdm
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

In [5]:
def process_pdf(file_path, chunk_size=800, chunk_overlap=50):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    # unique_paths = unique_paths[:2]
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr", 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


In [6]:
base_directory = './data' # Your Base Directory
df = pd.read_csv('./data/test.csv')
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


Processing 중소벤처기업부_혁신창업사업화자금(융자)...


Processing PDFs:  11%|█         | 1/9 [00:05<00:47,  5.97s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:09<00:32,  4.64s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:26<00:06,  3.41s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:29<00:03,  3.36s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:33<00:00,  3.72s/it]


# 모델 학습

개발 환경이 좋지 않아서 작은 모델로 실험

# Fine-Tune

In [4]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 모델 호출

개발환경의 문제로 양자화 & 16bit(모델 특성상 bf16)로 모델 호출

In [2]:
# 모델 ID 
model_id = "EleutherAI/polyglot-ko-1.3b"

# bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=torch.bfloat16
#     )

# 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드 및 양자화 설정 적용
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True )

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.13s/it]


# Dataset

train을 위한 dataset은 아래와 같은 형태로 필수 column이 존재함

따라서 적절한 tokenizer 함수를 통해 변환해 줘야함


In [3]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files='/home/csh/workspace/LLM_study/data/train.csv')

In [4]:
pd.read_csv('/home/csh/workspace/LLM_study/data/train.csv').columns

Index(['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer'], dtype='object')

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer'],
        num_rows: 496
    })
})

In [6]:
dataset = dataset['train'].train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer'],
        num_rows: 446
    })
    test: Dataset({
        features: ['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer'],
        num_rows: 50
    })
})

# Tokenizer

- **input_ids**

모델에 입력되는 문장의 각 토큰을 나타내는 정수 인덱스 -> 모델의 vocabulary와 매핑되어있음 

**token_type_ids**

문장 쌍을 처리할 때 각 문장을 구분하는 데 사용. 일반적으로 두 문장이 입력으로 주어지는 경우(예: 질문-응답 쌍, 문장 쌍 분류 등), 이 배열은 각 토큰이 어떤 문장에 속하는지를 나타냅니다.

**attention_mask**

패딩(padding) 토큰과 실제 입력 토큰을 구분하는 데 사용<br> : 1은 실제 입력 토큰 / 0은 패딩 토큰
모델은 패딩 토큰을 무시하고 실제 입력 토큰만을 처리

In [7]:
def tokenize_dataset(dataset):
    re_dataset = tokenizer(dataset['Question'], padding=True, truncation=True)
    label = tokenizer(dataset['Answer'], padding=True, truncation=True)
    re_dataset['labels'] = label['input_ids']
    return re_dataset

dataset = dataset.map(tokenize_dataset, batched=True)

Map: 100%|██████████| 50/50 [00:00<00:00, 2033.56 examples/s]


In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 446
    })
    test: Dataset({
        features: ['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
})

In [9]:
# 2로 패딩된거는 여기선 <|endoftext|>
tokenizer.decode(2)

'<|endoftext|>'

# LoRA

low rank로 변환해주는 과정

In [11]:
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers import DataCollatorForLanguageModeling

In [11]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2048, out_features=6144, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2048, out_features=8192, bias=True)
          (dense_4h_to_h): Linear(in_features=8192, out_features=2048, bias=True

### LoRA 적용 확인

In [12]:
from peft import get_peft_model

lora_config = LoraConfig(
    r=32,
    task_type="CAUSAL_LM"
)

p_model = get_peft_model(model, lora_config)
p_model.print_trainable_parameters()

trainable params: 6,291,456 || all params: 1,338,101,760 || trainable%: 0.4702


In [13]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Masked Language Model이 아닌 Causal Language Model의 경우 False로 설정
)

In [14]:
training_args = TrainingArguments(
    output_dir='./results',              # 결과를 저장할 디렉토리
    evaluation_strategy="steps",         # 평가 전략을 "steps"로 설정
    eval_steps=30,                      # 평가 간격 (스텝 단위)
    learning_rate=2e-5,                  # 학습률
    per_device_train_batch_size=4,       # 훈련 배치 크기
    per_device_eval_batch_size=4,        # 평가 배치 크기
    num_train_epochs=3,                  # 에폭 수
    weight_decay=0.01,                   # 가중치 감소
    logging_dir='./logs',                # 로그 디렉토리
    logging_steps=10,                    # 로그 기록 간격
    prediction_loss_only=True            # 생성 모델에서는 일반적으로 loss만을 예측
)

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Text Generate의 경우 SFT(Supervised-Fine-Tuning)Trainer 사용이 용이함

In [15]:
trainer = SFTTrainer(
    model=model,                         # 모델
    args=training_args,                  # 훈련 인수
    train_dataset=dataset['train'],   # 훈련 데이터셋
    eval_dataset=dataset['test'] ,  # 평가 데이터셋
    tokenizer=tokenizer,                 # 토크나이저
    data_collator=data_collator,           # 데이터 콜레이터
)

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [16]:
trainer.train()

Step,Training Loss,Validation Loss
30,2.990600,2.950000
60,2.751600,2.770000
90,2.740600,2.680625
120,2.218000,2.636875
150,2.356300,2.601875
180,2.180500,2.568125
210,2.305500,2.557500
240,2.098400,2.548750
270,2.182800,2.545000
300,2.371100,2.543750


TrainOutput(global_step=336, training_loss=2.4826543898809526, metrics={'train_runtime': 1403.1646, 'train_samples_per_second': 0.954, 'train_steps_per_second': 0.239, 'total_flos': 499663657156608.0, 'train_loss': 2.4826543898809526, 'epoch': 3.0})

# Inference

In [7]:
# 모델 ID 
model_id = "EleutherAI/polyglot-ko-1.3b"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

# 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델 로드 및 양자화 설정 적용
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True )

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:03<00:00,  1.23s/it]


In [8]:
# HuggingFacePipeline 객체 생성
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    return_full_text=False,
    max_new_tokens=128,
)

hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

/home/csh/workspace/LLM_study/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']

    # RAG 체인 구성
    template = """
    다음 정보를 바탕으로 질문에 답하세요:
    {context}

    질문: {question}

    답변:
    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | hf
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("./data/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("Nan")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("./data/baseline_submission.csv", encoding='UTF-8-sig', index=False)